### Imports

In [3]:
import numpy as np
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets

### Download the dataset

In [4]:
train_set = datasets.MNIST('/content',train=True,download=True)
test_set = datasets.MNIST('/content',train=False,download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /content/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /content/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /content/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /content/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/MNIST/raw



In [5]:
# inversion and normalization
invert = lambda image : 255 - image # function to invert the image

def preprocessing(dataset):
    dataset_images = dataset.data.numpy() # convert the dataset into numpy array
    dataset_labels = dataset.targets.numpy() # convert the labels into numpy array
    return dataset_images,dataset_labels

train_images,train_labels = preprocessing(train_set)
test_images,test_labels = preprocessing(test_set)

### Prepare the data

In [6]:
def generate_labels(dataset_images,image_type):
  labels = np.full(shape=(dataset_images.shape[0]),fill_value=[image_type])
  return labels

def binarize(dataset_images):
  result = np.copy(dataset_images)
  for i in range(result.shape[0]):
    for j in range(result.shape[1]):
      for k in range(result.shape[2]):
        if result[i][j][k] > 127:
          result[i][j][k] = 1
        else:
          result[i][j][k] = 0
  return result

# function for inverting data
def get_inverted_data(source_dataset_images,source_dataset_labels,target_dataset,class_num):
    image_indices = np.asarray(np.where(source_dataset_labels == class_num))
    image_indices = image_indices.flatten()
    original_images = invert(source_dataset_images[image_indices]) # get the original images and invert them
    target_dataset = np.concatenate((target_dataset,original_images)) # concatenate to our training dataset
    return target_dataset

# function to prepare training data for a given digit
def prepare_training_data(dataset_images,dataset_labels,class_num):
    indices = np.asarray(np.where(dataset_labels == class_num)) # indices of occurrence of digit as label
    indices = indices.flatten()
    # get the images for making positive dataset
    dataset_images_positive = dataset_images[indices] # images consisting of positive class
    dataset_images_positive = binarize(dataset_images_positive) # binarize the images 
    dataset_labels_positive = generate_labels(dataset_images_positive,0) # generate the class labels
    # print(f"Positive train data shape: {dataset_images_positive.shape}")
    # get the images for making negative dataset
    dataset_images_negative = np.empty((0,28,28),dtype=np.float32)
    dataset_images_negative = get_inverted_data(dataset_images,dataset_labels,dataset_images_negative,class_num)
    dataset_images_negative = binarize(dataset_images_negative) # binarize the images 
    dataset_labels_negative = generate_labels(dataset_images_negative,1) # generate the class labels
    # print(f"Negative train data shape: {dataset_images_negative.shape}")
    # concatenate the negative and positive datasets
    modified_dataset_images = np.concatenate((dataset_images_positive,dataset_images_negative))
    modified_dataset_labels = np.concatenate((dataset_labels_positive,dataset_labels_negative))
    return modified_dataset_images,modified_dataset_labels

def prepare_testing_data(dataset_images,dataset_labels,class_num):
  positive_indices = np.asarray(np.where(dataset_labels == class_num)) # indices of occurrence of digit as label
  positive_indices = positive_indices.flatten()
  # get the images for making positive dataset
  dataset_images_positive = dataset_images[positive_indices] # images consisting of positive class
  dataset_images_positive = binarize(dataset_images_positive)
  dataset_labels_positive = generate_labels(dataset_images_positive,0) # generate the class labels
  # print(f"Positive test data shape: {dataset_images_positive.shape}")
  # get the images for making negative classes for testing
  negative_indices = np.asarray(np.where(((dataset_labels != class_num))))
  negative_indices = negative_indices.flatten()
  # get the images for making negative dataset
  dataset_images_negative = dataset_images[negative_indices]
  dataset_images_negative = binarize(dataset_images_negative)
  dataset_labels_negative = generate_labels(dataset_images_negative,1)
  # print(f"Negative test data shape: {dataset_images_negative.shape}")
  # concatenate the negative and positive datasets
  modified_dataset_images = np.concatenate((dataset_images_positive,dataset_images_negative))
  modified_dataset_labels = np.concatenate((dataset_labels_positive,dataset_labels_negative))
  return modified_dataset_images,modified_dataset_labels

### Get training and test data

In [7]:
class_num = 0
trainX,trainy = prepare_training_data(train_images,train_labels,class_num)
testX,testy = prepare_testing_data(test_images,test_labels,class_num)

### Find patterns

In [32]:
patterns = np.empty((0,28,28),dtype=np.float32)

positive_image_indices = [i for i in range(trainX.shape[0]) if trainy[i] == 0]
positive_image = trainX[positive_image_indices[0]]
patterns = np.concatenate((patterns,[positive_image]))

accuracy = 0
while True:
  accuracy = 0
  for i in range(trainX.shape[0]):
    image,label = trainX[i],trainy[i]
    